In [1]:
import os 
import string
from numpy import loadtxt
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from collections import Counter

import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.pardir)
import glob
import numpy as np


from multiprocessing import Pool
import multiprocessing as multi
sys.path.append(os.pardir)
import pandas as pd
import h5py
import re
import gensim

# lines = loadtxt("/home/louis/SharedWindows/edu_data/"+"imdb-edus.train")
# train = []

# with open(dir+"imdb-edus.train", 'r') as train_file:
#     lines = train_file.readlines()


##



# '''text2index'''
# 
# from func import TextDeal
# from func import wrapper_func_four
# 
# vocab = wList
# 
# td = TextDeal()
# load_path = sorted( glob.glob( '/home/dl-box/デスクトップ/PythonFile/hirose/Text/text_list/*.txt' ) )
# name = [ os.path.splitext( os.path.basename( r ) )[0] for r in load_path ]
# save_path = '/home/dl-box/デスクトップ/PythonFile/hirose/LSTM/W2I/'



/home/owner/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/owner/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [56]:
def wrapper_func_six(tuple_data):
    return(tuple_data[0](tuple_data[1],tuple_data[2],tuple_data[3],tuple_data[4], tuple_data[5], tuple_data[6]) )

def readEduStyledFiles(dir, filename):
    with open(dir+filename, 'r') as train_file:
        lines = train_file.readlines()
    return lines
        
def tokenizeListOfTexts(list, vocab):
    idx = []
    isEmpty = False
    for word in list:
        if word in vocab:
            idx.append(vocab[word].index)
            #idx.append(len(word))
            #print(word)
        # else:
        #     print(word)
    if idx == []:
        isEmpty = True
    return idx, isEmpty

def addUniqueElementsToList(docs, li):
    for doc in docs:
        for sentence in doc:
            for word in sentence:
                if word not in li:
                    #print(word)
                    li.append(word)
    return li

def generateUniqueIndices(path_list):
    vocab_list = []
    for filepath in path_list:
        tmp = np.load(filepath)
        li = addUniqueElementsToList(tmp, vocab_list)
        vocab_list = vocab_list + li
    return vocab_list


def cleanAndTokenizeEDUs(vocab, load_path=None, load_name=None, save_path=None, save_name=None, willReturn = True, max_sent_len = 15,max_doc_len = 6):
    
    if (willReturn == False) and (all(v is None for v in [load_path, load_name, save_path, save_name])):
        raise ValueError('Function Arguments cannot all be None Type when saving files')
    elif (willReturn == True) and (all(v is None for v in [load_path, load_name])):
        raise ValueError('Load arguments cannot all be None Type when loading file')
    
    lines = readEduStyledFiles(load_path,load_name)

    cleaned_lines = []
    length = len(lines)
    scores = []
    ids = []
    cleaned_doc=[]
    cleaned_all=[]
    idx_binary = [] # index for binary training only
    scores_binary =[]
    
    doc_count = 0
    sent_counter = -1
    
    for i in range(len(lines)):  
        
        if len(cleaned_all) in [114,116,135]:
            a = 3
               
        sent_counter += 1
        
        cleaned = lines[i].replace('\n','').replace('<s>','').replace("'",'')
        cleaned = text_to_word_sequence(cleaned)
        
#         if i in [91,92,94]:
#             print(str(i)+' now enters')
#             print(cleaned)
#             print(sent_counter)
        
        if (len(cleaned_doc) <= max_doc_len):


#                 #DEBUGCODE
#                 if i in [91,92,94]:
#                     print(str(i)+' exit at len = 0 for sent<max')
                
            if (len(cleaned) == 2): # and (i != 0):
#                 #DEBUGCODE
#                 if i in [91,92,94]:
#                     print(str(i)+' exit at len = 2 for sent<max')
                
                if  (cleaned[0].isnumeric() and cleaned[1].isnumeric()):
                    
                    if i != 0:
                        if(len(cleaned_doc) < max_doc_len):
                            while (len(cleaned_doc) < max_doc_len):
                                tokenized = [0]
                                cleaned_doc.append(tokenized)
                    
                    score = int(cleaned[0])
                    id = int(cleaned[1])
                    scores.append(score)
                    ids.append(id)
                    
                    if (score<=4) or (score>=7):
                        idx_binary.append(doc_count)
                        if (score<4):
                            scores_binary.append(0)
                        else:
                            scores_binary.append(1)
                    
                    if (i != 0):
                        filtered_doc = list(filter(None, cleaned_doc))
                        padded_doc = pad_sequences(filtered_doc, maxlen=max_sent_len)

                        cleaned_all.append(padded_doc.tolist())
                        
                        #DEBUGCODE
                        if len(cleaned_all) in [114,116,135]:
                            print(str(len(cleaned_all))+' exit at len=2 for sent<=max') 
                        cleaned_doc=[]

                    doc_count += 1
                    sent_counter = 0
                   
                    continue
#             #DEBUGCODE        
#             if i in [91,92,94]:
#                 print(str(i)+' exit at all len for sent<max')
            if (len(cleaned_doc) <= max_doc_len-1):
                if cleaned != []:
                    tokenized, isEmpty =tokenizeListOfTexts(cleaned, vocab)
                    if not isEmpty:
                        cleaned_doc.append(tokenized)
            
        else:
            if (len(cleaned) == 2): # and (i != 0):
#                 #DEBUGCODE
#                 if i in [91,92,94]:
#                     print(str(i)+' exit at len = 2 for sent=>max')
                
                if  (cleaned[0].isnumeric() and cleaned[1].isnumeric()):
                    
                    score = int(cleaned[0])
                    id = int(cleaned[1])
                    scores.append(score)
                    ids.append(id)
                    
                    if (score<=4) or (score>=7):
                        idx_binary.append(doc_count)
                        if (score<4):
                            scores_binary.append(0)
                        else:
                            scores_binary.append(1)
                    
                    if (i != 0):
                        filtered_doc = list(filter(None, cleaned_doc))
                        padded_doc = pad_sequences(filtered_doc, maxlen=max_sent_len)
                        
                        cleaned_all.append(padded_doc.tolist())
                        
                        #DEBUGCODE
                        if len(cleaned_all) in [114,116,135]:
                            print(str(len(cleaned_all))+' exit at len=2 for sent>max') 
                            print(cleaned_doc)
                        cleaned_doc=[]
                        doc_count += 1
                        
                    sent_counter = 0
                    
#             #DEBUGCODE
#             if i in [91,92,94]:
#                 print(str(i)+' exit at all len for sent=>max') 
#                 print(sent_counter)
            
        if (i%1000000 == 0) and (i!=0):
            print(str(i)+' out of' + str(length))
            #DEBUG CODE
            #break
            # print(cleaned_lines[i-500:i])
            

    
    # Append the last doc 
    
    while (sent_counter < max_doc_len):
        tokenized = [0]
        cleaned_doc.append(tokenized)
        sent_counter += 1
        
    filtered_doc = list(filter(None, cleaned_doc))
    padded_doc = pad_sequences(filtered_doc, maxlen=max_sent_len)
    
    cleaned_all.append(padded_doc.tolist())
        
        # cleaned_doc = cleaned_doc[0]
    # 
    # del cleaned_all[0]
    # del scores[0]
    # del idx_binary[0]
    # del ids[0]
    features = np.asarray(cleaned_all)
    labels = np.asarray(scores)
    idx = np.asarray(idx_binary)
    
   
    
    if (willReturn == True):
        print('cleanAndTokenizeEDUs Returned and Done')
        return features, labels, idx, ids
    else:
        np.save(save_path + save_name + '_features'+ '.npy', features)
        np.save(save_path + save_name + '_labels' + '.npy', labels)
        np.save(save_path + save_name + '_idx_binary_only' + '.npy', idx)
        np.save(save_path + save_name + '_doc_ids' + '.npy', ids)
        np.save(save_path + save_name + '_scores_binary' + '.npy', scores_binary)


In [53]:
isReload = int(input("Type 1 to initiate reload of google word2vec: "))
    
if isReload:
    print('word2vec model now loading')
    model_word2vec_temp = gensim.models.KeyedVectors.load_word2vec_format('/home/owner/デスクトップ/milnet+edu/GoogleNews-vectors-negative300.bin', binary=True)  
    model_word2vec = model_word2vec_temp
    print('word2vec model loaded')

load_path = '/home/owner/デスクトップ/milnet+edu/data/'  
save_path = '/home/owner/デスクトップ/milnet+edu/data/Preprocessed/'

load_name = ['imdb-edus.train', 'imdb-edus.test','imdb-edus.dev']
save_name = ['train_data','test_data','validation_data']
name_trailers = ['_features','_labels','_idx_binary_only','_doc_ids']
willReturn = True

vocab = model_word2vec.vocab


features, scores, idx, ids = cleanAndTokenizeEDUs(vocab, load_path=load_path, load_name = 'imdb-edus.train', save_path=save_path,save_name=save_name[0], willReturn = willReturn)


i=0
#cleanAndTokenizeEDUs(model_word2vec.vocab, load_path=load_path, load_name = load_name[i], save_path=save_path,save_name=save_name[i], willSave = False)

Type 1 to initiate reload of google word2vec: 0
114 exit at len=2 for sent<=max
116 exit at len=2 for sent<=max
135 exit at len=2 for sent<=max


KeyboardInterrupt: 

In [ ]:
for i in range(len(features)):
    #print(len(features[i]))
    if len(features[i])!=6:
            print('i:'+str(i)+'   ;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;')
            print(len(features[i]))
    for j in range(len(features[i])):
        
        
        #print(len(features[i][j]))
        if len(features[i][j])!= 15:
            print('i:'+str(i)+'j:'+str(j)+'   ************************************************************')

In [ ]:
features

In [ ]:
lines = readEduStyledFiles(load_path,load_name[0])

In [ ]:
features[91]

In [57]:
if __name__ == '__main__':
    #data=(関数, 引数)
    isReload = int(input("Type 1 to initiate reload of google word2vec: "))
    
    if isReload:
        print('word2vec model now loading')
        model_word2vec_temp = gensim.models.KeyedVectors.load_word2vec_format('/home/owner/デスクトップ/milnet+edu/GoogleNews-vectors-negative300.bin', binary=True)  
        model_word2vec = model_word2vec_temp

    print('word2vec model loaded')

    load_path = '/home/owner/デスクトップ/milnet+edu/data/'  
    save_path = '/home/owner/デスクトップ/milnet+edu/data/Preprocessed/'
      
    load_name = ['imdb-edus.train', 'imdb-edus.test','imdb-edus.dev']
    save_name = ['train_data','test_data','validation_data']
    name_trailers = ['_features','_labels','_idx_binary_only','_doc_ids']
    willReturn = False
    

    #for i in range(len(load_name)):
    #i = 0
    #a,b,c,d = cleanAndTokenizeEDUs(model_word2vec.vocab, load_path=load_path, load_name = load_name[i], save_path=save_path,save_name=save_name[i], willSave = False)
    
    # features, scores, idx, ids = cleanAndTokenizeEDUs(model_word2vec.vocab, load_path=load_path, load_name = 'imdb-edus.train', save_path=save_path,save_name=save_name[0], willReturn = False)
    
    vocab = model_word2vec.vocab
    data = [ ( cleanAndTokenizeEDUs, vocab, load_path, load_name[i], save_path, save_name[i] , willReturn) for i in range( len(load_name) ) ]
    with Pool( multi.cpu_count()-1 ) as p:
        p.map( wrapper_func_six, data )
    


Type 1 to initiate reload of google word2vec: 0
word2vec model loaded
114 exit at len=2 for sent<=max
135 exit at len=2 for sent<=max
116 exit at len=2 for sent<=max
114 exit at len=2 for sent<=max
116 exit at len=2 for sent<=max
135 exit at len=2 for sent<=max
1000000 out of11039529
114 exit at len=2 for sent<=max
116 exit at len=2 for sent<=max
135 exit at len=2 for sent<=max
1000000 out of1349298
2000000 out of11039529
1000000 out of1335054
3000000 out of11039529
4000000 out of11039529
5000000 out of11039529
6000000 out of11039529
7000000 out of11039529
8000000 out of11039529
9000000 out of11039529
10000000 out of11039529
11000000 out of11039529


In [33]:
print('Starting Vocab List')
docs = [save_path+name+'_features'+'.npy' for name in save_name]
vocab_list = generateUniqueIndices(docs)
print('finished generating vocab')

Starting Vocab List
finished generating vocab


In [45]:
len(vocab_list)

421928

In [50]:
vocab_list = sorted(set(vocab_list))
np.save(save_path + 'vocab_idx.npy', vocab_list)

print('All Done')

All Done


In [51]:
np.save(save_path + 'weights.npy', model_word2vec.wv.syn0[vocab_list])    


In [17]:

train_features = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/train_data_features.npy')
valid_features = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/validation_data_features.npy')

test_features = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/test_data_features.npy')
test_labels = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/test_data_labels.npy')

In [39]:
print(len(train_features)+len(test_features)+len(valid_features))


NameError: name 'train_features' is not defined

In [19]:
output = set()
trends = [1,2,6,8,9,7,6,5,4,3,4,5,]
for x in trends:
    output.add(x)
print (output)

{1, 2, 3, 4, 5, 6, 7, 8, 9}


In [37]:
len(model_word2vec_temp.wv.syn0[vocab_list])
print(len(vocab_list))

421928


In [ ]:
vocab_list = np.asarray(vocab_list)
np.save(save_path + save_name + '_word_indices' + '.npy', vocab_list)